In [2]:
import pandas as pd

In [3]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")

tags_by_tag_old = client.get(data_as_key)[-1]['tags']

In [2]:
bert_data_key = ("ids_to_validate", "data", "bert_tags")
tags_data = client.get(bert_data_key)[-1]
labels_for_train = tags_data["gutenTag"]
print(len(labels_for_train))

254


In [3]:
# Check Tags In DICT
len([tags_by_tag_old[label] for label in labels_for_train])

254

In [4]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
from datetime import datetime
from collections import Counter
from itertools import permutations

from tqdm import tqdm


import pymongo
import bson
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [6]:
def dictImplementation(tagsList):
    if len(tagsList) > 0:
        result = []
        for tag in tagsList:
            if tag.lower() in tags_by_tag_old and tags_by_tag_old[tag.lower()] != [0]:
                result += tags_by_tag_old[tag.lower()]
            if tag.lower() not in tags_by_tag_old:
                result.append(tag.lower())
        return list(set(result))
    else:
        return []

In [8]:
GutenTagStats = {
}

allTagsRaw = []

for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"validated"}, 
                                                       {"status":"pipelined"}, {"status":"reserved"},
                                                       {"status":"graphicone_search"},{"status":"graphicone_feed"}, 
                                                       {"status": "deleted_from_analytics"}
                                                      ]
                                               }):

    guten_tag = []
    entity = []
    tagger = []
    analyst = []
    if "bert_tags" in i:
        if type(i["bert_tags"]) == list:
            guten_tag = i["bert_tags"]

    if "entity_prepared" in i:            
        if type(i["entity_prepared"]) == list:
            entity = i["entity_prepared"]
    #VALIDATED    
    if "confirmed_after_validate" in i:
        tagger = i["confirmed_after_validate"]
        
    #ANALYST
    if "grafeed_confirmed" in i:
        analyst = i["grafeed_confirmed"]

    try:
        status = i["status"]
        allModels = guten_tag + entity + tagger + analyst
        allModels = dictImplementation(allModels)
        allTagsRaw += allModels
        intersection = list(set(labels_for_train).intersection(set(allModels)))
        if len(intersection) > 0:
            if status not in GutenTagStats:
#                 print(status)
                GutenTagStats[status] = intersection
            else:
                GutenTagStats[status] += intersection
                
            
    except Exception as err:
        print(err)

'dict' object has no attribute 'lower'


In [9]:
GutenTagStats_2 = {}
for key in GutenTagStats.keys():
    print(key)
    GutenTagStats_2[key] = dict(Counter(GutenTagStats[key]).most_common())
    
GutenTagStats_2    

deleted_from_analytics
graphicone_feed
graphicone_search
pipelined
reserved
validated


{'deleted_from_analytics': {'commodities': 266,
  'gdp': 173,
  'labor': 170,
  'bonds': 163,
  'trade': 158,
  'global': 154,
  'manufacturing': 145,
  'social': 142,
  'developed markets': 139,
  'politics': 135,
  'mining': 135,
  'employment': 129,
  'retail': 127,
  'macro': 124,
  'consumer': 121,
  'metals': 114,
  'currency': 107,
  'real estate': 107,
  'large caps': 97,
  'government bonds': 96,
  'recession': 95,
  'energy': 94,
  'technology': 90,
  'emerging markets': 85,
  'etf': 80,
  'inflation': 78,
  'sentiment': 75,
  'financials': 73,
  'funds': 70,
  'government': 64,
  'banking': 62,
  'non-manufacturing': 60,
  'health': 59,
  'information technology': 53,
  'export': 52,
  'cryptocurrency': 51,
  'debt': 46,
  'services': 45,
  'consumer discretionary': 45,
  'consumer products': 45,
  'household': 43,
  'utilities': 42,
  'futures': 42,
  'internet': 41,
  'telecom': 39,
  'vehicles': 38,
  'taxes': 36,
  'imports': 36,
  'credit': 34,
  'industrials': 33,
  'h

In [10]:
feed = pd.DataFrame.from_dict(GutenTagStats_2["graphicone_feed"], orient="index").reset_index()
search = pd.DataFrame.from_dict(GutenTagStats_2["graphicone_search"], orient="index").reset_index()
dataForTrain = search.merge(feed, on = ["index"], how="outer", suffixes=('_search', '_feed'))
dataForTrain["SUM"] = dataForTrain["0_search"] + dataForTrain["0_feed"]
dataForTrain

,index,0_search,0_feed,SUM
0,global,734.0,132.0,866.0
1,commodities,567.0,109.0,676.0
2,technology,510.0,90.0,600.0
3,gdp,446.0,57.0,503.0
4,trade,427.0,53.0,480.0
5,labor,399.0,34.0,433.0
6,retail,379.0,56.0,435.0
7,social,367.0,50.0,417.0
8,macro,346.0,45.0,391.0
9,consumer,344.0,50.0,394.0


# NIL Categories

In [11]:
weakCategories = set(labels_for_train).difference(set(list(dataForTrain['index'].values)))
weakCategories

set()

In [12]:
def defineWeakLabels(x, minimum_level = 50):
    if pd.isnull(x["SUM"]) or x["SUM"] < minimum_level:
        return x.name

In [13]:
weakCategoriesSearchFeed = dataForTrain.set_index("index").apply(lambda x: defineWeakLabels(x), axis=1)
whiteList = list(weakCategoriesSearchFeed.dropna().values)+list(weakCategories)
print(len(whiteList))
print(set(whiteList))

145
{'waste', 'lifestyle', '3d printing', 'private equity', 'biomass', 'migration', 'obesity', 'crimes', 'bull market', 'bubble', 'unicorns', 'urbanization', 'student loan', 'culture', 'restaurants', 'trade war', 'fixed income', 'quantitative tightening', 'esports', 'life expectancy', 'quantitative easing', 'blockchain', 'sharing economy', 'crowdfunding', 'robotics', 'default', 'sustainability', 'car sharing', 'drugs', 'trading', 'biometrics', 'quantum computing', 'social justice', 'artificial intelligence', 'jewelry', 'infrastructure', 'chemical', 'geothermal', 'biotech', 'digital health', 'nonfarm payrolls', 'augmented reality', 'luxury', 'fashion', 'credit card', 'deflation', 'insider activity', 'shopping', 'wireless', 'wearables', 'apparel', 'mobile gaming', 'refugees', 'logistics', 'disasters', 'small caps', 'charity', 'brexit', 'high yield', 'delivery', 'tobacco', 'iot', 'semiconductors', 'terrorism', 'sports', 'poverty', 'subscriptions', 'volatility', 'pension funds', 'psycholog

In [14]:
pipelined = pd.DataFrame.from_dict(GutenTagStats_2["pipelined"], orient="index").reset_index()
pipelined

,index,0
0,global,846
1,commodities,818
2,social,719
3,technology,700
4,bonds,692
5,health,625
6,labor,570
7,politics,529
8,retail,501
9,real estate,438


In [15]:
AvailabelInPipe = pd.DataFrame(whiteList,columns = ["index"]).merge(pipelined, on=["index"], how="inner")
print(AvailabelInPipe.shape)
AvailabelInPipe

(130, 2)


,index,0
0,payments,117
1,cryptocurrency,212
2,m&a,83
3,social justice,95
4,trade war,97
5,dividends,150
6,drugs,109
7,children,110
8,fixed income,176
9,geopolitics,95


In [16]:
# list(AvailabelInPipe["index"].values)

In [17]:
def reserved(bson_id):
#     print("BEFORE:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["status"])
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                        {"$set":{"status":"reserved"}})
#     print("AFTER:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["status"])

In [18]:
def pipelined(bson_id):
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(bson_id)},
                                        {"$set":{"status":"pipelined"}})

In [19]:
db_client["tweets_pipeline_v2"].find({"status": "pipelined"}).count()

7777

In [20]:
db_client["tweets_pipeline_v2"].find({"status": "reserved"}).count()

18159

In [24]:
db = db_client["tweets_pipeline_v2"].find({"$or":[
#                                                     {"status": "pipelined"}, 
#                                                     {"status": "reserved"}
                                                    ]})
for i in tqdm(db, total = db.count()):
    guten_tag = []
    entity = []
    if "bert_tags" in i:
        guten_tag = i["bert_tags"]
    if "entity_prepared" in i: 
        entity = i["entity_prepared"]        
        
    allModels = guten_tag + entity
    allModels = dictImplementation(allModels)    
        
    if len(allModels) > 0 and len(set(allModels).intersection(set(whiteList))) > 0:
        pipelined(str(i["_id"]))
    else:
        reserved(str(i["_id"]))

100%|██████████| 20711/20711 [09:43<00:00, 35.52it/s]


In [21]:
db_client["tweets_pipeline_v2"].find({"status": "pipelined"}).count()

567

In [22]:
db_client["tweets_pipeline_v2"].find({"status": "reserved"}).count()

12998

In [7]:
db_client["tweets_pipeline_v2"].find({"status": "validated"}).count()

529

In [ ]:
def reserved(bson_id):
#     print("BEFORE:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["status"])

#     print("AFTER:", db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(bson_id)})["status"])

In [20]:
def to_temporaly_reserved(_id):
    status_before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)})["status"]
    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"temporaly_reserved",
                                            "status_before": status_before},
                                    })
#     print(_id)

In [24]:
def to_validated(_id):
    status_before = db_client["tweets_pipeline_v2"].find_one({"_id":bson.ObjectId(_id)})["status"]
    
    db_client["tweets_pipeline_v2"].update_one({"_id":bson.ObjectId(_id)},
                                    {"$set":{"status":"validated",
                                            "status_before": status_before},
                                    })
#     print(_id)

In [27]:
db = db_client["tweets_pipeline_v2"].find({"$or": [{"status":"temporaly_reserved"},                                                        
                                                      ]
                                               })
print(db.count())
counter = 0
for i in db:
    to_validated(str(i["_id"]))
#     tags = i["confirmed_after_validate"]
#     if "uk" in [tag.lower() for tag in tags]:
    counter +=1
#     else:
        
#         print()
counter

0


0

In [22]:
db = db_client["tweets_pipeline_v2"].find({"$or": [{"status":"validated"},                                                        
                                                      ],
                                           'validated_timestamp': {'$gte': datetime(2020, 4, 5)}
                                               },
                                          
                                         )

db.count()
for i in db:
    _id = str(i["_id"])
    to_temporaly_reserved(_id)

In [23]:
db_client["tweets_pipeline_v2"].find({"$or": [{"status":"temporaly_reserved"},                                                        
                                                      ]}).count()

181

In [ ]:
# 